<a href="https://colab.research.google.com/github/cccg8105/notebook_AD/blob/master/template%20Series%20temporales/AARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto ARIMA

## Instalación de dependencias

In [1]:
!pip install arch
!pip install pmdarima
!pip install yfinance

     |████████████████████████████████| 798kB 8.9MB/s 
     |████████████████████████████████| 1.5MB 8.9MB/s 
     |████████████████████████████████| 808kB 30.0MB/s 
     |████████████████████████████████| 8.7MB 31.9MB/s 
     |████████████████████████████████| 2.1MB 43.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 50.3.2
    Uninstalling setuptools-50.3.2:
      Successfully uninstalled setuptools-50.3.2
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


     |████████████████████████████████| 5.5MB 13.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=93417ddb62084a63183af7db2ffc2fe493b98e8fb28ec691938a31ff29b17710
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


## Importación de datos

In [11]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2020-03-20", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

df_comp = raw_data.copy()
df_comp.head()

[*********************100%***********************]  4 of 4 completed


^FTSE               ...         ^N225       
                   Open         High  ...         Close Volume
Date                                  ...                     
1994-01-07  3401.399902  3446.800049  ...  18124.009766    0.0
1994-01-10  3465.699951  3468.100098  ...  18443.439453    0.0
1994-01-11  3442.500000  3442.500000  ...  18485.250000    0.0
1994-01-12  3394.800049  3402.399902  ...  18793.880859    0.0
1994-01-13  3380.699951  3383.300049  ...  18577.259766    0.0

[5 rows x 20 columns]

### Formateo de datos

In [12]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')
df_comp.head()

,spx,dax,ftse,nikkei
,,,,
Date,,,,
1994-01-07,469.899994,2224.949951,3446.000000,18124.009766
1994-01-10,475.269989,2225.000000,3440.600098,18443.439453
1994-01-11,474.130005,2228.100098,3413.800049,18485.250000
1994-01-12,474.170013,2182.060059,3372.000000,18793.880859
1994-01-13,472.470001,2142.370117,3360.000000,18577.259766


## Retornos

In [13]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

## Auto ARIMA

In [15]:
from pmdarima.arima import auto_arima

model_auto = auto_arima(df_comp.ret_ftse[1:])
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 6834
Model:               SARIMAX(2, 0, 3)   Log Likelihood              -10391.632
Date:                Tue, 24 Nov 2020   AIC                          20795.264
Time:                        00:42:00   BIC                          20836.242
Sample:                             0   HQIC                         20809.401
                               - 6834                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.6587      0.067     -9.870      0.000      -0.790      -0.528
ar.L2         -0.3775      0.072     -5.221      0.000      -0.519      -0.236
ma.L1          0.6427      0.067      9.643      0.000       0.512       0.773
ma.L2          0.3307      0.072      4.583      0.000       0.189       0.472
ma.L3         -0.0864      0.007    -12.606      0.000      -0.100      -0.073
sigma2         1.2254      0.010    122.485      0.000       1.206       1.245
===================================================================================
Ljung-Box (Q):                       84.75   Jarque-Bera (JB):             15611.79
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.77   Skew:                            -0.31
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.38
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Modelo personalizado

In [16]:
model_auto2 = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample_size = int(len(df_comp)*0.2))


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [17]:
model_auto2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 6834
Model:             SARIMAX(0, 0, 1)x(4, 0, [], 5)   Log Likelihood               -6809.273
Date:                            Tue, 24 Nov 2020   AIC                          13640.547
Time:                                    01:28:09   BIC                          13715.673
Sample:                                         0   HQIC                         13666.465
                                           - 6834                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0008      0.015     -0.052      0.959      -0.031       0.029
drift      -2.401e-06   4.33e-06     -0.555      0.579   -1.09e-05    6.08e-06
x1             0.0872      0.006     15.219      0.000       0.076       0.098
x2             0.5624      0.005    110.721      0.000       0.552       0.572
x3             0.0720      0.004     16.452      0.000       0.063       0.081
ma.L1         -0.1165      0.008    -15.498      0.000      -0.131      -0.102
ar.S.L5       -0.0332      0.009     -3.776      0.000      -0.050      -0.016
ar.S.L10      -0.0503      0.010     -5.225      0.000      -0.069      -0.031
ar.S.L15      -0.0349      0.009     -3.844      0.000      -0.053      -0.017
ar.S.L20      -0.0233      0.009     -2.521      0.012      -0.041      -0.005
sigma2         0.4641      0.005     99.300      0.000       0.455       0.473
===================================================================================
Ljung-Box (Q):                       76.91   Jarque-Bera (JB):             15206.90
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.48   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.29
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""